In [0]:
! pip install SimpleITK flask-ngrok > /dev/null

In [0]:
import sys
import logging
from pathlib import Path
from flask import Flask, request
from flask_ngrok import run_with_ngrok

import SimpleITK as sitk
import numpy as np
import os
import tempfile
import random

os.chdir("drive/My Drive/6th Sem Project/BrainTumorSegmentation/data/HGG/Brats18_2013_26_1")

In [0]:
logging.basicConfig(
    format="[%(asctime)s %(levelname)s] %(message)s",
    datefmt="%d-%m-%Y %H:%M:%S",
    stream=sys.stdout,
    level=logging.DEBUG
)
logger = logging.getLogger()
app = Flask(__name__)
run_with_ngrok(app)
folder = Path(tempfile.gettempdir()) / "project-model-server"

@app.route("/api/upload", methods=["POST"])
def blah():
  logger.info("Received request")
  file_dir: Path = folder / str(random.randint(0, 1000))
  Path.mkdir(file_dir, parents=True)

  for item in request.files.items():
    item[1].save(str(file_dir / f"{item[0]}.nii.gz"))
  
  logger.info("Saved files. Testing...")
  test(file_dir, "t1")
  test(file_dir, "t1ce")
  test(file_dir, "t2")
  test(file_dir, "flair")
  test(file_dir, "seg")
  

  return {}, 200

def load(path: str):
  return sitk.GetArrayFromImage(sitk.ReadImage(path))

def compare(data1, data2):
  return np.all(data1 == data2)

def test(file_dir, tp):
  t_data = load(str(file_dir/f"{tp}.nii.gz"))
  o_data = load(f"Brats18_2013_26_1_{tp}.nii.gz")
  logger.info("%s test: %s", tp, compare(t_data, o_data))

In [0]:
from app import app

In [3]:
app.run()

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[19-05-2020 03:49:32 INFO]  * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[19-05-2020 03:49:33 DEBUG] Starting new HTTPS connection (1): bin.equinox.io:443
[19-05-2020 03:49:33 DEBUG] https://bin.equinox.io:443 "GET /c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip HTTP/1.1" 200 13773305
[19-05-2020 03:49:35 DEBUG] Starting new HTTP connection (1): localhost:4040
[19-05-2020 03:49:35 DEBUG] http://localhost:4040 "GET /api/tunnels HTTP/1.1" 200 771
 * Running on http://106f74ff.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
[19-05-2020 03:51:12 INFO] Received request
[19-05-2020 03:51:13 INFO] Saved files. Testing...
[19-05-2020 03:51:16 INFO] t1 test: True
[19-05-2020 03:51:18 INFO] t1ce test: True
[19-05-2020 03:51:20 INFO] t2 test: True
[19-05-2020 03:51:22 INFO] flair test: True
[19-05-2020 03:51:23 INFO] seg test: True
[19-05-2020 03: